In [193]:
import PyPDF2
import pandas as pd
import re

In [194]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        text = ""
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

In [195]:
# Provide the path to your PDF file
pdf_path = r'C:\Users\AKhairunnisa\Dev-Python\Part C - General Terms and Conditions.pdf'

# Call the function to extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_path)


In [196]:
# Split the extracted text into rows and columns
rows = extracted_text.split('\n')
table_data = [row.split('\t') for row in rows]

In [197]:
# Convert the table data to a DataFrame
df = pd.DataFrame({'ExtractedText':table_data[0:]})

In [198]:
# Replace some strings
df['ExtractedText'] = df['ExtractedText'].astype(str)
df['ExtractedText'] = df['ExtractedText'].str.replace('[', '')
df['ExtractedText'] = df['ExtractedText'].str.replace(']', '')
df['ExtractedText'] = df['ExtractedText'].str.replace(chr(39), '')

In [199]:
df['Length'] = df['ExtractedText'].apply(len)

if (df['Length'] == 1).any():
    df.loc[df['Length'] == 1, 'Category'] = 'Blank'
else:
    df['Category'] = 'Not Blank'



In [200]:
# Check if extracted test contains "Page"
df['ContainsPage'] = df['ExtractedText'].str.contains('Page')

if (df['ContainsPage'] == True).any():
    df.loc[df['ContainsPage'], 'PageNo'] = df.loc[df['ContainsPage'], 'ExtractedText'].str[5:7]
else:
    df['PageNo'] = None
# Fill down previous values in 'PageNo'
df['PageNo'] = df['PageNo'].fillna(method='ffill')


In [201]:
df = df.fillna('0')
daftar_isi = df.loc[df['Category'] != 'Blank']

In [202]:
df['PageNo'] = df['PageNo'].astype(int)
daftar_isi = df.loc[df['PageNo'] < 9]

In [203]:
# Regex pattern for "number.number" with named capture group
pattern = r"(?P<SubSectionNo>\d+\.\d+)"

# Create a new column 'ExtractedValues' with extracted patterns
daftar_isi['SubSectionNo'] = daftar_isi['ExtractedText'].str.extract(pattern, expand=False)

# Replace non-matching values with blank
daftar_isi['SubSectionNo'].fillna('', inplace=True)

C:\Users\AKhairunnisa\AppData\Local\Temp\ipykernel_21692\2828521014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daftar_isi['SubSectionNo'] = daftar_isi['ExtractedText'].str.extract(pattern, expand=False)
C:\Users\AKhairunnisa\AppData\Local\Temp\ipykernel_21692\2828521014.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daftar_isi['SubSectionNo'].fillna('', inplace=True)


In [204]:
# Regex pattern for "number.number" with named capture group
pattern = r'\d+\.\s'

# Create a new column 'NewColumn' to extract the pattern if matched, otherwise leave it blank
daftar_isi['SectionNo'] = daftar_isi['ExtractedText'].str.extract(f'({pattern})', expand=False).fillna('')

C:\Users\AKhairunnisa\AppData\Local\Temp\ipykernel_21692\3356370221.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daftar_isi['SectionNo'] = daftar_isi['ExtractedText'].str.extract(f'({pattern})', expand=False).fillna('')


In [205]:
# Extract the value based on delimiter from 'Column1' using value from 'DelimiterColumn'
daftar_isi['SectionName'] = daftar_isi.apply(lambda row: row['ExtractedText'].split(row['SubSectionNo'])[1] if row['SubSectionNo'] != '' and row['SubSectionNo'] in row['ExtractedText'] else row['SubSectionNo'], axis=1)


C:\Users\AKhairunnisa\AppData\Local\Temp\ipykernel_21692\671591702.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daftar_isi['SectionName'] = daftar_isi.apply(lambda row: row['ExtractedText'].split(row['SubSectionNo'])[1] if row['SubSectionNo'] != '' and row['SubSectionNo'] in row['ExtractedText'] else row['SubSectionNo'], axis=1)


In [206]:
daftar_isi['SectionName'] = daftar_isi.apply(lambda row: row['ExtractedText'].split(row['SectionNo'])[1] if row['SectionNo'] != '' and row['SectionNo'] in row['ExtractedText'] else row['SectionName'], axis=1)


C:\Users\AKhairunnisa\AppData\Local\Temp\ipykernel_21692\726570132.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daftar_isi['SectionName'] = daftar_isi.apply(lambda row: row['ExtractedText'].split(row['SectionNo'])[1] if row['SectionNo'] != '' and row['SectionNo'] in row['ExtractedText'] else row['SectionName'], axis=1)
